# Prepare rooftop masks

In [1]:
import os
import re

import numpy as np

import pandas as pd
import geopandas as gpd

import rasterio.features
import rasterio.transform

from PIL import Image

from tqdm import tqdm

In [2]:
# Load the whole rooftop dataset
path = "../data/solkat/SOLKAT_DACH.gpkg"
gdf = gpd.read_file(
    path,
    layer="SOLKAT_CH_DACH",
    columns=[],
    engine="pyogrio",
    use_arrow=True,
)

In [5]:
# List files that must be generated
input_folder = "../data/sample/1"
output_folder = "../data/mask/1"
names = [name for name in os.listdir(input_folder) if name.endswith(".jpg")]
pairs = []
for name in names:
    output_path = os.path.join(output_folder, name[:-4] + ".jpg")
    if not os.path.exists(output_path):
        match = re.match(r"swissimage-dop10_\d+_(\d+)\.(\d)-(\d+)\.(\d)", name)
        assert match
        i = int(match.group(1))
        j = int(match.group(3))
        u = int(match.group(2))
        v = int(match.group(4))
        pair = (i, j, u, v), output_path
        pairs.append(pair)

In [6]:
# Generate masks
for (i, j, u, v), output_path in tqdm(pairs):
    x_min = i * 1000 + u * 100
    x_max = i * 1000 + (u + 1) * 100
    y_min = j * 1000 + v * 100
    y_max = j * 1000 + (v + 1) * 100
    chunk_gdf = gdf.cx[x_min:x_max, y_min:y_max]
    transform = rasterio.transform.from_bounds(x_min, y_min, x_max, y_max, 1000, 1000)
    array = rasterio.features.rasterize(
        chunk_gdf.geometry,
        transform=transform,
        out_shape=(1000, 1000),
        fill=0,
        default_value=255,
        dtype=np.uint8,
    )
    image = Image.fromarray(array)
    image.save(output_path)

  0%|          | 0/250 [00:00<?, ?it/s]

100%|██████████| 250/250 [37:29<00:00,  9.00s/it]
